In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap[all]
# !{sys.executable} -m pip install pip rasterstats 

In [1]:
import ee
# ee.Authenticate()

In [2]:
ee.Initialize()

In [3]:
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import io
from rasterstats import zonal_stats
import fiona
import rasterio.mask
import geemap
import glob
import boto3

In [4]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

# Read input data

In [11]:
## Read Land use land cover dataset
WC = ee.ImageCollection("ESA/WorldCover/v100")
WorldCover = WC.first();

## define projection for use later
WCprojection = WC.first().projection();  
print('WorldCover projection:', WCprojection.getInfo());

WorldCover projection: {'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [8.333333333333333e-05, 0, -180, 0, -8.333333333333333e-05, 84]}


In [12]:
# define directory
out_dir = os.getcwd()
aws_s3_dir = "https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data"

In [15]:
# get list of c4f cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')
boundary_georef

,city_name,geo_name,aoi_boundary_name,units_boundary_name,city_boundary_name,country_code,geo_level
0,Salvador,BRA-Salvador,ADM4union,ADM4,BRA-Salvador-ADM4,BRA,ADM4
1,Bukavu,COD-Bukavu,ADM3union,ADM3,COD-Bukavu-ADM3,COD,ADM3
2,Uvira,COD-Uvira,ADM3union,ADM3,COD-Uvira-ADM3,COD,ADM3
3,Brazzaville,COG-Brazzaville,ADM4union,ADM4,COG-Brazzaville-ADM4,COG,ADM4
4,Barranquilla,COL-Barranquilla,ADM4union,ADM4,COL-Barranquilla-ADM4,COL,ADM4
5,Addis_Ababa,ETH-Addis_Ababa,ADM4union,ADM4,ETH-Addis_Ababa-ADM4,ETH,ADM4
6,Dire_Dawa,ETH-Dire_Dawa,ADM3union,ADM3,ETH-Dire_Dawa-ADM3,ETH,ADM3
7,Nairobi,KEN-Nairobi,ADM3union,ADM3,KEN-Nairobi-ADM3,KEN,ADM3
8,Antananarivo,MDG-Antananarivo,ADM4union,ADM4,MDG-Antananarivo-ADM4,MDG,ADM4
9,Mexico_City,MEX-Mexico_City,ADM2union,ADM2,MEX-Mexico_City-ADM2,MEX,ADM2


# Compute indicator

In [16]:
cities_indicators_GRE_1_3 = pd.DataFrame() 

In [17]:
# set parameters for albedo calculations: date range of interest, image limit, scale, and albedo threshold

date_start = '2021-01-01'
date_end = '2022-01-01'
image_limit = 50 # max number of images to include, sorted from least to most cloudy
scale = 10 # scale in meters at which to complete reductions - 10 by default, increase if experiencing timeouts with large geographies

# define "low albedo" threshold
LowAlbedoMax = 0.20 # EnergyStar steep slope minimum initial value is 0.25. 3-year value is 0.15. https://www.energystar.gov/products/building_products/roof_products/key_product_criteria


In [18]:
## Configure methods

# Read relevant Sentinel-2 data
S2 = ee.ImageCollection("COPERNICUS/S2_SR")
S2C = ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")

MAX_CLOUD_PROB=30
S2_ALBEDO_EQN='((B*Bw)+(G*Gw)+(R*Rw)+(NIR*NIRw)+(SWIR1*SWIR1w)+(SWIR2*SWIR2w))'
S2_VIZ = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3};


## METHODS

## get cloudmasked image collection 

def mask_and_count_clouds(s2wc,geom):
    s2wc=ee.Image(s2wc)
    geom=ee.Geometry(geom.geometry())
    is_cloud=ee.Image(s2wc.get('cloud_mask')).gt(MAX_CLOUD_PROB).rename('is_cloud')
    nb_cloudy_pixels=is_cloud.reduceRegion(
        reducer=ee.Reducer.sum().unweighted(), 
        geometry=geom, 
        scale=10, 
        maxPixels=1e9
   )
    return s2wc.updateMask(is_cloud.eq(0)).set('nb_cloudy_pixels',nb_cloudy_pixels.getNumber('is_cloud')).divide(10000)

def mask_clouds_and_rescale(im):
    clouds=ee.Image(im.get('cloud_mask')).select('probability')
    return im.updateMask(clouds.lt(MAX_CLOUD_PROB)).divide(10000)

def get_masked_s2_collection(roi,start,end):
    criteria=(ee.Filter.And(
            ee.Filter.date(start,end),
            ee.Filter.bounds(roi)
        ))
    s2=S2.filter(criteria)#.select('B2','B3','B4','B8','B11','B12')
    s2c=S2C.filter(criteria)
    s2_with_clouds=(ee.Join.saveFirst('cloud_mask').apply(**{
        'primary': ee.ImageCollection(s2),
        'secondary': ee.ImageCollection(s2c),
        'condition': ee.Filter.equals(**{'leftField':'system:index','rightField':'system:index'}) 
        }))
    def _mcc(im):
        return mask_and_count_clouds(im,roi) 
    #s2_with_clouds=ee.ImageCollection(s2_with_clouds).map(_mcc)
    #s2_with_clouds=s2_with_clouds.limit(image_limit,'nb_cloudy_pixels')
    s2_with_clouds=ee.ImageCollection(s2_with_clouds).map(mask_clouds_and_rescale)#.limit(image_limit,'CLOUDY_PIXEL_PERCENTAGE')
    return  ee.ImageCollection(s2_with_clouds)

# calculate albedo for images

# weights derived from 
# S. Bonafoni and A. Sekertekin, "Albedo Retrieval From Sentinel-2 by New Narrow-to-Broadband Conversion Coefficients," in IEEE Geoscience and Remote Sensing Letters, vol. 17, no. 9, pp. 1618-1622, Sept. 2020, doi: 10.1109/LGRS.2020.2967085.
def calc_s2_albedo(image):
    config={
    'Bw':0.2266,
    'Gw':0.1236,
    'Rw':0.1573,
    'NIRw':0.3417,
    'SWIR1w':0.1170,
    'SWIR2w':0.0338,
    'B':image.select('B2'),
    'G':image.select('B3'),
    'R':image.select('B4'),
    'NIR':image.select('B8'),
    'SWIR1':image.select('B11'),
    'SWIR2':image.select('B12')
  }
    return image.expression(S2_ALBEDO_EQN,config).float().rename('albedo')


In [19]:
for i in range(0,1):#,len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']

        
    # process aoi level ------
    print("\n boundary_id_aoi: "+boundary_id_aoi)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
      
    ## S2 MOSAIC AND ALBEDO
    dataset = get_masked_s2_collection(boundary_geo_ee,date_start,date_end)
    s2_albedo = dataset.map(calc_s2_albedo)
    mosaic=dataset.mean()
    albedoMean=s2_albedo.reduce(ee.Reducer.mean())
    albedoMeanThres = albedoMean.updateMask(albedoMean.lt(LowAlbedoMax))
    

    # define images and functions

    ## function to create image of means of toCount for each asClass
    def getmeanbyclass(classvalue):
        return ee.Image(toCount.updateMask(asClass.eq(classvalue)) #.And(toCount.gt(0))) # uncomment And statement if you want include only pixels that meet both criteria
                        # .unmask(0) # uncomment if you want to include all pixels not just pixels of classvalue
                        ).rename(ee.String('') #'class_count-'
                                           .cat(ee.Number(classvalue).toInt().format()))

    ## function to create image of count of each asClass
    def getcountbyclass(classvalue):
        return ee.Image(toCount.updateMask(asClass.eq(classvalue)) #.And(toCount.gt(0))) # uncomment And statement if you want include only pixels that meet both criteria
                        # .unmask(0) # uncomment if you want to include all pixels not just pixels of classvalue
                        ).rename(ee.String('class_count-') #'class_count-'
                                          .cat(ee.Number(classvalue).toInt().format()))

    ## function to create image of count of each asClass filtered by toCount
    def getcountbyclassFilt(classvalue):
        return ee.Image(toCount.updateMask(asClass.eq(classvalue).And(toCount.gte(0))) # uncomment And statement if you want include only pixels that meet both criteria
                        # .unmask(0) # uncomment if you want to include all pixels not just pixels of classvalue
                        ).rename(ee.String('class_countFilt-') #'class_count-'
                                          .cat(ee.Number(classvalue).toInt().format()))

    ## create image with each WorldCover class mean as a band

    asClass = WorldCover
    toCount = albedoMean 

    # meanbyclass=ee.Image(getmeanbyclass(10)).addBands([
    #   getmeanbyclass(20),  
    #   getmeanbyclass(30),  
    #   getmeanbyclass(40),  
    #   getmeanbyclass(50),  
    #   getmeanbyclass(60),
    #   getmeanbyclass(70),  
    #   getmeanbyclass(80), 
    #   getmeanbyclass(90), 
    #   getmeanbyclass(95), 
    #   getmeanbyclass(100), 
    # ])

    ## create image with each WorldCover class count as a band

    countbyclass=ee.Image(getcountbyclass(10)).addBands([
      getcountbyclass(20),  
      getcountbyclass(30),  
      getcountbyclass(40),  
      getcountbyclass(50),  
      getcountbyclass(60),
      getcountbyclass(70),  
      getcountbyclass(80), 
      getcountbyclass(90), 
      getcountbyclass(95), 
      getcountbyclass(100), 
    ])

    ## create image with each WorldCover class count above threshold as a band
    toCount = albedoMeanThres 

    countbyclassFilt=ee.Image(getcountbyclassFilt(10)).addBands([
      getcountbyclassFilt(20),  
      getcountbyclassFilt(30),  
      getcountbyclassFilt(40),  
      getcountbyclassFilt(50),  
      getcountbyclassFilt(60),
      getcountbyclassFilt(70),  
      getcountbyclassFilt(80), 
      getcountbyclassFilt(90),
      getcountbyclassFilt(95), 
      getcountbyclassFilt(100), 
    ])

            
    def reducers(FC):
        ## create FeatureCollection with mean of count for each class for each feature

        histo=countbyclass.reduceRegions(
          reducer= ee.Reducer.count(), 
          collection= FC, 
          scale= scale, 
          tileScale= 4
        )

        histo=countbyclassFilt.reduceRegions(
          reducer= ee.Reducer.count(), 
          collection= histo, 
          scale= scale, 
          tileScale= 4
        )
        
        # histo=meanbyclass.reduceRegions(
        #   reducer= ee.Reducer.mean(), 
        #   collection= histo, 
        #   scale= 10, 
        #   tileScale= 4
        # )
        return histo
    
    
    ## Define function to normalize count as percent of all pixels in each feature and create new properties with the values
    
    def count_to_percent(feat):
        feat=ee.Feature(feat)
        
#         hist=ee.Dictionary(feat.toDictionary(['10','20','30','40','50','60','70','80','90','95','100']))
#         hist=hist.set('10',hist.get('10',0))
#         hist=hist.set('20',hist.get('20',0))
#         hist=hist.set('30',hist.get('30',0))
#         hist=hist.set('40',hist.get('40',0))
#         hist=hist.set('50',hist.get('50',0))
#         hist=hist.set('60',hist.get('60',0))
#         hist=hist.set('70',hist.get('70',0))
#         hist=hist.set('80',hist.get('80',0))
#         hist=hist.set('90',hist.get('90',0))
#         hist=hist.set('95',hist.get('95',0))
#         hist=hist.set('100',hist.get('100',0))

#         def pct_hist(k,v):
#             # convert whole number (0-100) to decimal percent (0-1)
#             return ee.Number(v)

#         meansLULC = hist.map(pct_hist)

        histC=ee.Dictionary(feat.toDictionary(['class_count-10','class_count-20','class_count-30','class_count-40','class_count-50','class_count-60','class_count-70','class_count-80','class_count-90','class_count-95','class_count-100']))
        histC=histC.set('50',histC.get('class_count-50',0))
        # histC=histC.set('10',histC.get('class_count-10',0))
        # histC=histC.set('20',histC.get('class_count-20',0))
        # histC=histC.set('30',histC.get('class_count-30',0))
        # histC=histC.set('40',histC.get('class_count-40',0))
        # histC=histC.set('60',histC.get('class_count-60',0))
        # histC=histC.set('70',histC.get('class_count-70',0))
        # histC=histC.set('80',histC.get('class_count-80',0))
        # histC=histC.set('90',histC.get('class_count-90',0))
        # histC=histC.set('95',histC.get('class_count-95',0))
        # histC=histC.set('100',histC.get('class_count-100',0))

        histCfilt=ee.Dictionary(feat.toDictionary(['class_countFilt-10','class_countFilt-20','class_countFilt-30','class_countFilt-40','class_countFilt-50','class_countFilt-60','class_countFilt-70','class_countFilt-80','class_countFilt-90','class_countFilt-95','class_countFilt-100']))
        histCfilt=histCfilt.set('50',histCfilt.get('class_countFilt-50',0))
        # histCfilt=histCfilt.set('10',histCfilt.get('class_countFilt-10',0))
        # histCfilt=histCfilt.set('20',histCfilt.get('class_countFilt-20',0))
        # histCfilt=histCfilt.set('30',histCfilt.get('class_countFilt-30',0))
        # histCfilt=histCfilt.set('40',histCfilt.get('class_countFilt-40',0))
        # histCfilt=histCfilt.set('60',histCfilt.get('class_countFilt-60',0))
        # histCfilt=histCfilt.set('70',histCfilt.get('class_countFilt-70',0))
        # histCfilt=histCfilt.set('80',histCfilt.get('class_countFilt-80',0))
        # histCfilt=histCfilt.set('90',histCfilt.get('class_countFilt-90',0))
        # histCfilt=histCfilt.set('95',histCfilt.get('class_countFilt-95',0))
        # histCfilt=histCfilt.set('100',histCfilt.get('class_countFilt-100',0))

        def area_hist(k,v):
            # convert 10m pixel count of class to KM2 of class
            return ee.Number(v).multiply(ee.Number(100)).multiply(ee.Number(0.000001))

        classAreas = histC.map(area_hist)
        classFiltAreas = histCfilt.map(area_hist)

        # totalPixels=hist.values()

        return feat.set({
            # 'LC10albedo': meansLULC.getNumber('10'),
            # 'LC20albedo': meansLULC.getNumber('20'),
            # 'LC30albedo': meansLULC.getNumber('30'),
            # 'LC40albedo': meansLULC.getNumber('40'),
            # 'LC50albedo': meansLULC.getNumber('50'),
            # 'LC60albedo': meansLULC.getNumber('60'),
            # 'LC70albedo': meansLULC.getNumber('70'),
            # 'LC80albedo': meansLULC.getNumber('80'),
            # 'LC90albedo': meansLULC.getNumber('90'),
            # 'LC95albedo': meansLULC.getNumber('95'),
            # 'LC100albedo': meansLULC.getNumber('100'),
            # 'LC10lowAlbedoPct': classFiltAreas.getNumber('10').divide(classAreas.getNumber('10')),
            # 'LC20lowAlbedoPct': classFiltAreas.getNumber('20').divide(classAreas.getNumber('20')),
            # 'LC30lowAlbedoPct': classFiltAreas.getNumber('30').divide(classAreas.getNumber('30')),
            # 'LC40lowAlbedoPct': classFiltAreas.getNumber('40').divide(classAreas.getNumber('40')),
            'LC50lowAlbedoPct': classFiltAreas.getNumber('50').divide(classAreas.getNumber('50')),
            # 'LC60lowAlbedoPct': classFiltAreas.getNumber('60').divide(classAreas.getNumber('60')),
            # 'LC70lowAlbedoPct': classFiltAreas.getNumber('70').divide(classAreas.getNumber('70')),
            # 'LC80lowAlbedoPct': classFiltAreas.getNumber('80').divide(classAreas.getNumber('80')),
            # 'LC90lowAlbedoPct': classFiltAreas.getNumber('90').divide(classAreas.getNumber('90')),
            # 'LC95lowAlbedoPct': classFiltAreas.getNumber('95').divide(classAreas.getNumber('95')),
            # 'LC100lowAlbedoPct': classFiltAreas.getNumber('100').divide(classAreas.getNumber('100')),
        })

    
    ## update FeatureCollection with percents
    histo = reducers(FC = boundary_geo_ee)
    albedo_means=histo.map(count_to_percent).select(['geo_id','LC50lowAlbedoPct'])
    
    # store in df and append
    df = geemap.ee_to_pandas(albedo_means)
    df = df.rename(columns={"LC50lowAlbedoPct": "GRE_1_3_percentBuiltwLowAlbedo"})
    cities_indicators_GRE_1_3 = cities_indicators_GRE_1_3.append(df)
    
    
    # process unit of analysis level ------
    print("\n boundary_id_unit: "+boundary_id_unit)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_unit+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    ## update FeatureCollection with percents
    histo = reducers(FC = boundary_geo_ee)
    albedo_means=histo.map(count_to_percent).select(['geo_id','LC50lowAlbedoPct'])
    
    # store in df and append
    df = geemap.ee_to_pandas(albedo_means)
    df = df.rename(columns={"LC50lowAlbedoPct": "GRE_1_3_percentBuiltwLowAlbedo"})
    cities_indicators_GRE_1_3 = cities_indicators_GRE_1_3.append(df)


0

 geo_name: BRA-Salvador

 boundary_id_aoi: BRA-Salvador-ADM4union

 boundary_id_unit: BRA-Salvador-ADM4


In [ ]:
cities_indicators_GRE_1_3

# Merge with indicator table

In [ ]:
# read indicator table
cities_indicators = pd.read_csv(aws_s3_dir + '/indicators/cities_indicators_erictest.csv') 
cities_indicators#.head()

In [ ]:
def merge_indicators(indicator_table, new_indicator_table, indicator_name):
    if indicator_name in indicator_table.columns:
        print("replace by new indicators")
        indicator_table.drop(indicator_name, inplace=True, axis=1)
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    else:
        print("add new indicators")
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    return(cities_indicators_df)

In [ ]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators,
                                            new_indicator_table = cities_indicators_GRE_1_3,
                                            indicator_name = "GRE_1_3_percentBuiltwLowAlbedo")

In [ ]:
cities_indicators_merged

# Upload in aws s3

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
# upload to aws
key_data = 'data/indicators/cities_indicators_erictest.csv'
bucket_name = 'cities-cities4forests' 
cities_indicators_merged.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [ ]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')